<a href="https://colab.research.google.com/github/Pakhi27/Vechile-Dataset-Detection-YOLO./blob/main/Vechile_Dataset_Detection_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Convert Annotations to YOLO Format
# First, you need to convert the CSV annotations to YOLO format.
import os

# Define the paths
# base_image_dir: Path where vehicle images are stored.
# base_annotation_dir: Path where corresponding annotation files are located.
# output_dir: The directory where converted YOLO-format annotations will be saved.
base_image_dir = '/content/drive/MyDrive/archive (43)'
base_annotation_dir = '/content/drive/MyDrive/archive (43)/annotation'
output_dir = '/content/labels_yolo/'  # Directory for YOLO formatted annotations

# Vehicle types and their class IDs
class_mapping = {
    'van': 0,
    'bus': 1,
    'auto':2,
    'tractor':3
}
# This dictionary maps vehicle types (van, bus, auto, and tractor) to unique numeric class IDs, which is required by YOLO for training.

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
# This creates the output_dir if it doesn't exist.

# Function to copy annotations to YOLO folder
# This function takes in a vehicle_type (e.g., 'van', 'bus', etc.) and processes images and annotations for that type.
def organize_annotations_and_images(vehicle_type):
    image_dir = os.path.join(base_image_dir, vehicle_type)
    annotation_dir = os.path.join(base_annotation_dir, vehicle_type)
    # These lines define the paths to the vehicle-specific image and annotation folders.
    # This loop iterates through all image files (JPG, JPEG, PNG) in the image directory.
    for image_file in os.listdir(image_dir):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_name = os.path.splitext(image_file)[0]
            annotation_file = os.path.join(annotation_dir, f"{image_name}.txt")
            # The annotation file for each image is located using its filename (without the extension) and appending ".txt" to it.
            # Move the annotation file to the YOLO format folder
            if os.path.exists(annotation_file):
                output_annotation_file = os.path.join(output_dir, f"{image_name}.txt")
                os.rename(annotation_file, output_annotation_file)

                # Copy the image to the YOLO training folder
                output_image_file = os.path.join(base_image_dir, f"{vehicle_type}/{image_file}")
                os.rename(os.path.join(image_dir, image_file), output_image_file)

                print(f"Organized {image_file} and its annotation")
# If the annotation file exists, it is moved to the output_dir. The image is also moved to the correct directory for YOLO.
# Organize annotations and images for all vehicle types
for vehicle_type in ['van', 'bus','auto','tractor']:
    organize_annotations_and_images(vehicle_type)
# This calls the function for each vehicle type: van, bus, auto, and tractor.
print("All annotations and images organized.")




All annotations and images organized.


In [11]:
#  Create YOLOv5 Configuration File
yaml_content = """
train: '/content/drive/MyDrive/archive (43)'  # Path to the folder with training images
val: '/content/drive/MyDrive/archive (43)'    # Using the same folder for validation (if no separate validation set)
nc: 4  # Number of classes (van, bus, auto, tractor)
names: ['van', 'bus', 'auto', 'tractor']  # List of class names
"""

# Write the YAML content to a file
with open('/content/vehicle.yaml', 'w') as file:
    file.write(yaml_content)


print("vehicle.yaml file created.")

vehicle.yaml file created.


In [13]:
!git clone https://github.com/ultralytics/yolov5.git
# This command clones the official YOLOv5 repository from GitHub.

Cloning into 'yolov5'...
remote: Enumerating objects: 16954, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 16954 (delta 76), reused 95 (delta 45), pack-reused 16805 (from 1)
Receiving objects: 100% (16954/16954), 15.71 MiB | 17.75 MiB/s, done.
Resolving deltas: 100% (11609/11609), done.


In [14]:
#  Train the YOLOv5 Model
# This command starts training the YOLOv5 model using the provided data:
# --img 640: Resizes input images to 640x640 pixels.
# --batch 16: Uses a batch size of 16.
# --epochs 50: Runs the training for 50 epochs.
# --data: The path to the vehicle.yaml file that contains class and data paths.
# --weights: Uses the pre-trained YOLOv5 small (yolov5s.pt) weights as the starting point.
!python /content/yolov5/train.py --img 640 --batch 16 --epochs 50 --data /content/vehicle.yaml --weights yolov5s.pt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.6/871.6 kB 14.4 MB/s eta 0:00:00
2024-09-14 17:43:24.602648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 17:43:24.863466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 17:43:24.879869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
requirements: Ultralytics requirement ['gitpython'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [23]:
import os
import cv2

folder = '/content/drive/MyDrive/archive (43)'

# Loop through all the subdirectories and files
for subdir, dirs, files in os.walk(folder):
    for file in files:
        # Check if the file is an image by its extension
        if file.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(subdir, file)
            try:
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Could not read the image file {img_path}")
                else:
                    print(f"Successfully read image {img_path}")
            except Exception as e:
                print(f"Error reading {img_path}: {e}")


Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (1).jpg
Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (10).jpg
Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (12).jpg
Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (13).jpg
Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (15).jpg
Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (11).jpg
Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (14).jpg
Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (16).jpg
Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (17).jpg
Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (24).jpg
Successfully read image /content/drive/MyDrive/archive (43)/van/Datacluster_van (19).jpg
Successfully read imag

KeyboardInterrupt: 

In [31]:
print(model, img_pil)

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [36]:
!python /content/yolov5/train.py --img-size 640 --batch-size 16 --epochs 50 --data vehicle.yaml --cfg yolov5s.yaml --weights '' --name vehicle_model


2024-09-14 18:38:37.831614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 18:38:37.866017: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 18:38:37.878093: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=, cfg=yolov5s.yaml, data=vehicle.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, sing

In [40]:
import os
import cv2
import torch
from PIL import Image
import numpy as np

# Load the custom-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5s.pt')  # Update the path as needed

# Path to the main folder containing all subfolders
main_folder = '/content/drive/MyDrive/archive (43)'
output_folder = '/content/output_images_folder/'  # Folder to save output images with detections

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
print(f"Output folder created at: {output_folder}")

# List of folders to process
folders_to_process = ['van', 'bus', 'tractor', 'auto']

# Loop through each specified folder
for folder_name in folders_to_process:
    folder_path = os.path.join(main_folder, folder_name)

    # Check if the folder exists
    if not os.path.isdir(folder_path):
        print(f"Folder {folder_path} does not exist. Skipping.")
        continue

    # Create a subfolder in the output folder for the current folder
    output_subfolder = os.path.join(output_folder, folder_name)
    os.makedirs(output_subfolder, exist_ok=True)
    print(f"Output subfolder created at: {output_subfolder}")

    # Loop through all images in the folder
    for image_name in os.listdir(folder_path):
        if image_name.endswith(('.jpg', '.jpeg', '.png')):  # Check for image files
            image_path = os.path.join(folder_path, image_name)

            try:
                # Read the image using OpenCV
                img = cv2.imread(image_path)
                if img is None:
                    print(f"Could not read the image file {image_path}")
                    continue

                # Convert the image from BGR (OpenCV format) to RGB (PIL format)
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_pil = Image.fromarray(img_rgb)

                # Perform object detection
                results = model(img_pil)

                # Print detection results
                results.print()

                # Render results and convert them to numpy arrays
                img_output = results.render()[0]

                # Convert the result image from RGB to BGR format
                img_output_bgr = cv2.cvtColor(img_output, cv2.COLOR_RGB2BGR)

                # Save the output image with bounding boxes
                output_image_path = os.path.join(output_subfolder, image_name)
                cv2.imwrite(output_image_path, img_output_bgr)

                print(f"Processed {image_name} in folder {folder_name} and saved to {output_image_path}")

            except Exception as e:
                print(f"Error processing {image_name} in folder {folder_name}: {e}")

print("Object detection complete for all images.")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-14 Python-3.10.12 torch-2.4.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Output folder created at: /content/output_images_folder/
Output subfolder created at: /content/output_images_folder/van


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 3000x4000 1 bird
Speed: 42.5ms pre-process, 494.7ms inference, 2.5ms NMS per image at shape (1, 3, 480, 640)


Processed Datacluster_van (1).jpg in folder van and saved to /content/output_images_folder/van/Datacluster_van (1).jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 3008x4016 1 person, 2 clocks
Speed: 62.1ms pre-process, 513.7ms inference, 1.8ms NMS per image at shape (1, 3, 480, 640)


...

Processed Datacluster_auto (8).jpg in folder auto and saved to /content/output_images_folder/auto/Datacluster_auto (8).jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 3120x4160 3 persons, 1 boat, 1 traffic light
Speed: 46.1ms pre-process, 436.6ms inference, 2.0ms NMS per image at shape (1, 3, 480, 640)


Processed Datacluster_auto (9).jpg in folder auto and saved to /content/output_images_folder/auto/Datacluster_auto (9).jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 3120x4160 1 traffic light
Speed: 26.5ms pre-process, 317.9ms inference, 1.4ms NMS per image at shape (1, 3, 480, 640)


Processed Datacluster_auto (7).jpg in folder auto and saved to /content/output_images_folder/auto/Datacluster_auto (7).jpg
Object detection complete for all images.
